In [1]:
import socket
import json
import ast
import threading
import select
import queue
import pandas as pd
import numpy as np

In [2]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time

In [3]:
class SaveAcquisition(object):
    def __init__(self):
        self.fileTxt = None
        self.hasHeader = False
 
    def start(self):
        self.fileTxt = open("tcp_py3_Acquisition.txt", "w")
 
    def addData(self, data):
        self.fileTxt.write(data)
        self.hasHeader = True
 
    def stop(self):
        self.fileTxt.close()
 
    def getHasHeader(self):
        return self.hasHeader
 

In [4]:
class DataBuffer(object):
    def __init__(self):
        self.data = []
        self.isFull = False
        self.size = 5000

dbuff = DataBuffer()

In [5]:
class TCPClient(object):
    def __init__(self):
        # self.tcpIp = ''
        self.tcpPort = 3333
        self.tcpIp = 'localhost'
 
        self.buffer_size = 99999
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.inputCheck = []
        self.outputCheck = []
        self.isChecking = False
        self.isAcquiring = False
        self.msgQueue = queue.Queue()
 
        self.txtFile = SaveAcquisition()
 
    def connect(self):
        self.socket.connect((self.tcpIp, self.tcpPort))
        self.outputCheck.append(self.socket)
        self.isChecking = True
 
    def start(self):
        print("XX ", self.isChecking)
        thread = threading.Thread(target=self.msgChecker)
        thread.daemon = True
        thread.start()
 
    def stop(self):
        self.isChecking = False
        self.socket.close()
 
    def msgChecker(self):
        global dbuff
        while self.isChecking:
            readable, writable, exceptional = select.select(self.inputCheck, self.outputCheck, self.inputCheck)
            
            
            for s in readable:
                message = s.recv(self.buffer_size)
                if not self.isAcquiring:
                    print(message)
                    self.inputCheck = []
                else:
                    print(message)
                    message = json.loads(message.decode('utf8'))
                    message = message["returnData"]
                    if not self.txtFile.getHasHeader():
                        newLine = json.dumps(message) + "\n"
                        self.txtFile.addData(newLine)
                    else:
                        dataframe = []
                        for device in list(message.keys()):
                            dataframe.append(pd.DataFrame(message[device]))
                        dataframe = pd.concat(dataframe, axis=1, ignore_index=True)
                        for line in dataframe.values:
                            self.txtFile.addData('\n')
                            self.txtFile.addData(",".join([str(x) for x in line]))
#                             if not dbuff.isFull:
#                                 if len(dbuff.data) < dbuff.size:
#                                     dbuff.data.append([float(x) for x in line])
#                                 else:
                                    
                                
#                             else:
                                
#                                 dbuff.data.append([float(x) for x in line])
                            
 
            for s in writable:
                try:
                    next_msg = self.msgQueue.get_nowait()
                except queue.Empty:
                    pass
                else:
                    print("send ")
                    self.socket.send(next_msg.encode('utf-8'))
 
            for s in exceptional:
                print("exceptional ", s)
 
    def addMsgToSend(self, data):
        self.msgQueue.put(data)
        if self.socket not in self.outputCheck:
            self.outputCheck.append(self.socket)
        if self.socket not in self.inputCheck:
            self.inputCheck.append(self.socket)
 
    def setIsAcquiring(self, isAcquiring):
        self.isAcquiring = isAcquiring
        if self.isAcquiring:
            self.txtFile = SaveAcquisition()
            self.txtFile.start()
        else:
            self.txtFile.stop()
            
    def collectBuffer(self, isAcquiring):
        if self.isAcquiring:
            self.txtFile = SaveAcquisition()
            self.txtFile.start()
        else:
            self.txtFile.stop()

In [6]:
# CONNECTION = TCPClient()
# CONNECTION.connect()
# t0 = time.time()
# CONNECTION.setIsAcquiring(True)
# new_msg = 'start'
# CONNECTION.start()
# CONNECTION.addMsgToSend(new_msg)

# while True:
#     te = time.time() - t0

#     if (te > 10 ):
#         new_msg = 'stop'
#         CONNECTION.setIsAcquiring(False)
#         CONNECTION.addMsgToSend(new_msg)


#     if (te > 11 ):
#         new_msg = ''
#         CONNECTION.stop()
#         CONNECTION.addMsgToSend(new_msg)

#         break
# print("END")

In [7]:
CONNECTION = TCPClient()
CONNECTION.connect()
t0 = time.time()


In [8]:
CONNECTION.setIsAcquiring(True)
new_msg = 'start'
CONNECTION.start()
CONNECTION.addMsgToSend(new_msg)

XX  True
send 
b'{"returnData": {}, "returnCode": 0}'
b'{"returnData": {"20:16:07:18:13:60": [[12.0, 0.0, 0.0, 0.0, 0.0, 1.202, 24.22], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [0.0, 0.0, 0.0, 0.0, 0.0, 1.173, 24.12], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [3.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.41], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.22], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.12], [9.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.22], [13.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.22], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [15.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.32], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.22], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24

b'{"returnData": {"20:16:07:18:13:60": [[14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [15.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.22], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.12], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 23.93], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 23.93], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.22], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [6.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.32], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.22], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.32], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [2.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.22], [5.0, 0.0

b'{"returnData": {"20:16:07:18:13:60": [[10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.02], [11.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.02], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.02], [13.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.22], [14.0, 0.0, 0.0, 0.0, 0.0, 1.192, 23.93], [15.0, 0.0, 0.0, 0.0, 0.0, 1.192, 23.93], [0.0, 0.0, 0.0, 0.0, 0.0, 1.173, 24.02], [1.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.22], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 24.02], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [6.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.02], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [8.0, 0.0, 0.0, 0.0, 0.0, 1.173, 24.02], [9.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.22], [10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.02], [11.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.02], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.02], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.02], [1.0,

b'{"returnData": {"20:16:07:18:13:60": [[6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [8.0, 0.0, 0.0, 0.0, 0.0, 1.154, 24.12], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.02], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.12], [1.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.02], [2.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.02], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [6.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [8.0, 0.0, 0.0, 0.0, 0.0, 1.173, 24.22], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.32], [12.0, 0.0, 0.0, 0.0, 0.0, 1.202, 24.12], [13.0, 0

b'{"returnData": {"20:16:07:18:13:60": [[2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 24.32], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.02], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.32], [9.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [12.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.32], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.22], [1.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.12], [2.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 24.32], [5.0, 0.0, 0.0, 0.0, 0.0, 1.192, 24.22], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.22], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 24.12], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 24.22], [9.0, 0.0, 

...

b'{"returnData": {"20:16:07:18:13:60": [[8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.78], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.07], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [12.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.88], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.17], [14.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [15.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.78], [1.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.98], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.78], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.88], [5.0, 0.0, 0.0, 0.0, 0.0, 1.192, 26.17], [6.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.98], [7.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.88], [9.0, 0.0, 0.0, 0.0, 0.0, 1.192, 26.07], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.78], [13.0, 0.0, 0.0, 0.0, 0.0, 1.192, 26.07], [14.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [15.0,

b'{"returnData": {"20:16:07:18:13:60": [[4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.78], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.07], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.07], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.98], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.98], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.98], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.07], [12.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.88], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.07], [0.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.98], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [2.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.98], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.17], [4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.88], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 26.07], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.78], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [11.0, 0.0

b'{"returnData": {"20:16:07:18:13:60": [[0.0, 0.0, 0.0, 0.0, 0.0, 1.154, 26.07], [1.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.78], [2.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 26.17], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 26.07], [9.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.78], [10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.78], [11.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.78], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 26.07], [13.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [14.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.98], [15.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [0.0, 0.0, 0.0, 0.0, 0.0, 1.173, 26.17], [1.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.88], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [3.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.78], [4.0, 0.0, 0.0, 0.0, 0.0, 1.221, 26.07], [5.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.88], [6.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.88], [7.0, 0.0, 

b'{"returnData": {"20:16:07:18:13:60": [[12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.68], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [0.0, 0.0, 0.0, 0.0, 0.0, 1.154, 25.78], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [2.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.68], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.68], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.98], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.78], [12.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.78], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.88], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.68], [0.0, 0.0, 0.0, 0.0, 0.0, 1.154, 25.68], [1.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.68], [2.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.88], [3.0, 0

b'{"returnData": {"20:16:07:18:13:60": [[8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.29], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.29], [10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.59], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.49], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.49], [14.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.59], [0.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.39], [1.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.49], [4.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.39], [5.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [6.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.49], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.59], [8.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.49], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [10.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.49], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.39], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.29], [14.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [15.0,

In [9]:
new_msg = 'stop'
CONNECTION.setIsAcquiring(False)
CONNECTION.addMsgToSend(new_msg)

b'{"returnData": {"20:16:07:18:13:60": [[4.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.29], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [6.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.29], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.49], [9.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.49], [10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [11.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [12.0, 0.0, 0.0, 0.0, 0.0, 1.212, 25.29], [13.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [14.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.29], [15.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.29], [0.0, 0.0, 0.0, 0.0, 0.0, 1.173, 25.39], [1.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.49], [2.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [3.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.39], [4.0, 0.0, 0.0, 0.0, 0.0, 1.202, 25.39], [5.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.49], [6.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.29], [7.0, 0.0, 0.0, 0.0, 0.0, 1.183, 25.29], [8.0, 0.0, 0.0, 0.0, 0.0, 1.163, 25.29], [9.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.39], [10.0, 0.0, 0.0, 0.0, 0.0, 1.192, 25.29], [11.0, 0.0

In [10]:
dbuff.data

[]

In [ ]:
CONNECTION.stop()

In [ ]:
#ak 192.168.0.106
ip_tcp = "192.168.1.85"
port = 32000

In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time

deflating = False

def print_volume_handler(unused_addr, args, volume):
    global deflating, deflateStartTime
    
    print("[{0}] ~ {1}".format(args, volume))
    
    if (volume > 1090 or deflating):
        client.send_message("/actuator/1/inflate", -80.0)   # Send float message between -100 and 100
        
        if deflating == False:
            deflating = True
            deflateStartTime = time.time()
        
        elif time.time() - deflateStartTime > 5:
            deflating = False
            
    elif deflating == False:
        client.send_message("/actuator/1/inflate", 80.0)   # Send float message between -100 and 100


deflateStartTime = time.time()
    
ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/sensor/pressure", print_volume_handler, "Pressure")

server = osc_server.ThreadingOSCUDPServer(("192.168.0.106", 31000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
from pythonosc.udp_client import SimpleUDPClient

ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client

client.send_message("/actuator/1/inflate", -0.0)


In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient


def print_volume_handler(unused_addr, args, volume):
    print("[{0}] ~ {1}".format(args, volume))
    
    client.send_message("/actuator/1/inflate", -100.0)   # Send float message between -100 and 100


    
ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/sensor/pressure", print_volume_handler, "Pressure")

server = osc_server.ThreadingOSCUDPServer(("192.168.0.106", 31000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
import time
time.time()